# Sending Requests
This notebook provides a quick-start guide to use SGLang in chat completions after installation.

- For Vision Language Models, see [OpenAI APIs - Vision](../backend/openai_api_vision.ipynb).
- For Embedding Models, see [OpenAI APIs - Embedding](../backend/openai_api_embeddings.ipynb) and [Encode (embedding model)](../backend/native_api.html#Encode-(embedding-model)).
- For Reward Models, see [Classify (reward model)](../backend/native_api.html#Classify-(reward-model)).

## Launch A Server

In [1]:
from sglang.test.test_utils import is_in_ci
from sglang.utils import wait_for_server, print_highlight, terminate_process

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

# This is equivalent to running the following command in your terminal

# python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0

server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct \
 --host 0.0.0.0
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-07 00:51:12] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=31259, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=117955845, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1,

[2025-05-07 00:51:28] Attention backend not set. Use fa3 backend by default.
[2025-05-07 00:51:28] Init torch distributed begin.


[2025-05-07 00:51:29] Init torch distributed ends. mem usage=0.00 GB
[2025-05-07 00:51:29] Load weight begin. avail mem=51.27 GB


[2025-05-07 00:51:29] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-07 00:51:30] Using model weights format ['*.safetensors']


...

Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.38it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.38it/s]

[2025-05-07 00:51:31] Load weight end. type=Qwen2ForCausalLM, dtype=torch.bfloat16, avail mem=50.20 GB, mem usage=1.06 GB.
[2025-05-07 00:51:31] KV Cache is allocated. #tokens: 20480, K size: 0.12 GB, V size: 0.12 GB
[2025-05-07 00:51:31] Memory pool end. avail mem=49.80 GB


[2025-05-07 00:51:32] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768


[2025-05-07 00:51:33] INFO:     Started server process [1627450]
[2025-05-07 00:51:33] INFO:     Waiting for application startup.
[2025-05-07 00:51:33] INFO:     Application startup complete.
[2025-05-07 00:51:33] INFO:     Uvicorn running on http://0.0.0.0:31259 (Press CTRL+C to quit)


[2025-05-07 00:51:33] INFO:     127.0.0.1:41910 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-07 00:51:34] INFO:     127.0.0.1:41918 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-07 00:51:34] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


## Using cURL


In [2]:
import subprocess, json

curl_command = f"""
curl -s http://localhost:{port}/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{{"model": "qwen/qwen2.5-0.5b-instruct", "messages": [{{"role": "user", "content": "What is the capital of France?"}}]}}'
"""

response = json.loads(subprocess.check_output(curl_command, shell=True))
print_highlight(response)

[2025-05-07 00:51:38] Prefill batch. #new-seq: 1, #new-token: 36, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-05-07 00:51:38] INFO:     127.0.0.1:41924 - "POST /generate HTTP/1.1" 200 OK
[2025-05-07 00:51:38] The server is fired up and ready to roll!


[2025-05-07 00:51:39] Decode batch. #running-req: 1, #token: 76, token usage: 0.00, gen throughput (token/s): 7.11, #queue-req: 0
[2025-05-07 00:51:39] INFO:     127.0.0.1:41926 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using Python Requests

In [3]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "qwen/qwen2.5-0.5b-instruct",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-05-07 00:51:39] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 35, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-07 00:51:39] INFO:     127.0.0.1:41934 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using OpenAI Python Client

In [4]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)
print_highlight(response)

[2025-05-07 00:51:39] Prefill batch. #new-seq: 1, #new-token: 13, #cached-token: 24, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-07 00:51:39] Decode batch. #running-req: 1, #token: 59, token usage: 0.00, gen throughput (token/s): 64.20, #queue-req: 0


[2025-05-07 00:51:40] INFO:     127.0.0.1:41940 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Streaming

In [5]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

# Use stream=True for streaming responses
response = client.chat.completions.create(
    model="qwen/qwen2.5-0.5b-instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
    stream=True,
)

# Handle the streaming output
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

[2025-05-07 00:51:40] INFO:     127.0.0.1:41956 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-05-07 00:51:40] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 36, token usage: 0.00, #running-req: 0, #queue-req: 0
Sure

,

 here

 are

 three

...

 **

Australia

**

 -

 Canberra

## Using Native Generation APIs

You can also use the native `/generate` endpoint with requests, which provides more flexiblity. An API reference is available at [Sampling Parameters](sampling_params.md).

In [6]:
import requests

response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "The capital of France is",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 32,
        },
    },
)

print_highlight(response.json())

[2025-05-07 00:51:40] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-07 00:51:41] Decode batch. #running-req: 1, #token: 29, token usage: 0.00, gen throughput (token/s): 70.24, #queue-req: 0
[2025-05-07 00:51:41] INFO:     127.0.0.1:41968 - "POST /generate HTTP/1.1" 200 OK


### Streaming

In [7]:
import requests, json

response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "The capital of France is",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 32,
        },
        "stream": True,
    },
    stream=True,
)

prev = 0
for chunk in response.iter_lines(decode_unicode=False):
    chunk = chunk.decode("utf-8")
    if chunk and chunk.startswith("data:"):
        if chunk == "data: [DONE]":
            break
        data = json.loads(chunk[5:].strip("\n"))
        output = data["text"]
        print(output[prev:], end="", flush=True)
        prev = len(output)

[2025-05-07 00:51:41] INFO:     127.0.0.1:41972 - "POST /generate HTTP/1.1" 200 OK
[2025-05-07 00:51:41] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 4, token usage: 0.00, #running-req: 0, #queue-req: 0
 Paris

.

 It

 is

 the

...

 on

 the

 banks

 of

[2025-05-07 00:51:41] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, gen throughput (token/s): 74.23, #queue-req: 0
 the

In [8]:
terminate_process(server_process)

[2025-05-07 00:51:41] Child process unexpectedly failed with an exit code 9. pid=1627756
